<a href="https://colab.research.google.com/github/mazinkamal134/DS_2024_MRP/blob/main/Master_Dataset_Processing_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Ingest the tweets
- Add the diagnosis dates and treatment dates
- Add the music sessions
- Add the TensiStrengh stress scores
- Add the user info

In [ ]:
import pandas as pd
import pickle
from datetime import datetime
import os
import json

In [2]:
# mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Global Params

In [ ]:
tweetsDir = "/content/drive/MyDrive/MRP/Mazin_Works/Project Docs/Data/Tweets"
tensiStrenghtDir = "/content/drive/MyDrive/MRP/Mazin_Works/Project Docs/Data/TensiStrenght"
demographicsDir = "/content/drive/MyDrive/MRP/Mazin_Works/Project Docs/Data/Demographics"
authorsDir = "/content/drive/MyDrive/MRP/Mazin_Works/Project Docs/Data/Authors"
musicDir = r"/content/drive/MyDrive/MRP/Mazin_Works/Project Docs/Data/Music"

In [ ]:
# Read parquet file for modifications only
# Otherwise, skip this step and go through all other steps one by one
"""
fileName = os.path.join(tweetsDir, "processedTweets.parquet")
tweetsDf = pd.read_parquet(fileName)
"""

### Ingest the control tweets

In [ ]:
# Read the control Tweets CSV file and pickle
fileName = os.path.join(tweetsDir, "control_tweets.csv")
controlTweetsDf = pd.read_csv(fileName)
# Add the group
controlTweetsDf["group"] = 0
print("Control Shape:", controlTweetsDf.shape)

Control Shape: (6452058, 19)


### Ingest the treatment tweets

In [ ]:
# Read the treatment Tweets CSV file and pickle
fileName = os.path.join(tweetsDir, "treatment_tweets.csv")
treatmentTweetsDf = pd.read_csv(fileName)
# Add the group
treatmentTweetsDf["group"] = 1
print("Treatment Shape:", treatmentTweetsDf.shape)

Treatment Shape: (9090297, 19)


### Combine control and treatment

In [ ]:
# Combine
tweetsDf = pd.concat([controlTweetsDf, treatmentTweetsDf])
print("Shape combined:", tweetsDf.shape)

# Free up the menmory (only on Jupytor, no need on Google Colab with High-RAM)
#del controlTweetsDf
#del treatmentTweetsDf

# Fix the data types
tweetsDf["created_at"] = pd.to_datetime(tweetsDf.created_at).dt.tz_convert(None)
tweetsDf["author_id"] = tweetsDf["author_id"].astype("str")

# Reorder the columns
cols = ["id", "tweet_type", "referenced_tweet_type", "created_at", "lang", "disorder", "author_id", "text", "cleaned_text", "retweet_count", "reply_count", "like_count", "quote_count", "source", "group"]
tweetsDf = tweetsDf[cols]

Shape combined: (15542355, 19)


### Get the diagnose dates


In [ ]:
diagnosisDatesDf = tweetsDf[tweetsDf.tweet_type == "diagnose"].reset_index(drop = True)
# Convert the created_at to date
diagnosisDatesDf["created_at"] = diagnosisDatesDf["created_at"].dt.date
# Keep only the author_id, created_at, and group, and rename created_at to diagnosis_date
diagnosisDatesDf = diagnosisDatesDf[["author_id", "created_at"]].rename(columns = {"created_at": "diagnosis_date"})
# Select the tweet with the minimum created data for each author and remove others
diagnosisDatesDf = diagnosisDatesDf.groupby("author_id").diagnosis_date.min().reset_index()
print ("Diagnosis Dates Shape:", diagnosisDatesDf.shape)
diagnosisDatesDf.sample()

Diagnosis Dates Shape: (8624, 2)


,author_id,diagnosis_date
2295,1251994534470397952,2020-06-24


#### Update the diagnosis dates for all

In [ ]:
# Add the diagnosis date to the main dataframe
tweetsDf = tweetsDf.merge(diagnosisDatesDf, on = ["author_id"], how = "left")
print("Shape after updating with diagnosis date:", tweetsDf.shape)
# Count the unmatched records
print("Unmatched:", tweetsDf[tweetsDf.diagnosis_date.isna()].shape[0])
# Check
tweetsDf.sample()

Shape after updating with diagnosis date: (15542355, 16)
Unmatched: 0


,id,tweet_type,referenced_tweet_type,created_at,lang,disorder,author_id,text,cleaned_text,retweet_count,reply_count,like_count,quote_count,source,group,diagnosis_date
11843367,1353096844461666304,timeline,replied_to,2021-01-23 21:46:41,en,depression,701871080001167360,@enhypeus @sianbffs someone got into a fs and ...,someone got into a fs and didn’t even know the...,0,1,1,0,Twitter for iPhone,1,2020-09-24


### Find and populate the treatment dates

In [ ]:
# Add the anchor dates
datesDir = r"/content/drive/MyDrive/MRP/Mazin_Works/Project Docs/Data/Dates"
controlFilePath = os.path.join(datesDir, "control_users_features_summary.json")
treatmentFilePath = os.path.join(datesDir, "treatment_users_features_summary.json")

# Read the json data
with open(controlFilePath, "rb") as f:
    file = json.load(f)
controlDf = pd.DataFrame(file).transpose().reset_index()
controlDf["Group"] = 0
with open(treatmentFilePath, "rb") as f:
    file = json.load(f)
treatmentDf = pd.DataFrame(file).transpose().reset_index()
treatmentDf["Group"] = 1

# Combine
treatmentDatesDf = pd.concat([controlDf, treatmentDf], ignore_index = True)

# Process
treatmentDatesDf.rename(columns = {"index": "author_id", "diagnose_date": "treatment_date"}, inplace = True)

# drop unnecessary columns
colsToKeep = ["author_id", "treatment_date", "Group"]
treatmentDatesDf = treatmentDatesDf[colsToKeep]

# Fix the data types
treatmentDatesDf["treatment_date"] = pd.to_datetime(treatmentDatesDf["treatment_date"])

# Drop duplicates
treatmentDatesDf.drop_duplicates()

# Check
print("Shape:", treatmentDatesDf.shape)
# Select the author with the minimum treatment date
treatmentDatesDf = treatmentDatesDf.groupby(["author_id"]).treatment_date.min().reset_index()
treatmentDatesDf.sample()

Shape: (8606, 3)


,author_id,treatment_date
6103,388436664,2021-05-06


In [ ]:
# Join the treatment dates with the main dataframe
tweetsDf = tweetsDf.merge(treatmentDatesDf, on = ["author_id"], how = "left")
print("Shape after adding the treatment dates", tweetsDf.shape)
# Check
tweetsDf.sample()

Shape after adding the treatment dates (15542355, 17)


,id,tweet_type,referenced_tweet_type,created_at,lang,disorder,author_id,text,cleaned_text,retweet_count,reply_count,like_count,quote_count,source,group,diagnosis_date,treatment_date
4254330,1431484812288741379,timeline,replied_to,2021-08-28 05:12:29,en,depression,3838035553,"cant help but to compare, sobrang lunod ako sa...","cant help but to compare, sobrang lunod ako sa...",0,0,0,0,Twitter for Android,0,2020-07-06,2021-12-01


### Add the music sessions

In [ ]:
# Ingest the music file
musicFilePath = os.path.join(musicDir, "music.csv")
musicDf = pd.read_csv(musicFilePath)
# Fix the data types
musicDf["created_at"] = pd.to_datetime(musicDf["created_at"]).dt.tz_convert(None)
musicDf["author_id"] = musicDf["author_id"].astype("str")
# rename tweet_id to id
musicDf.rename(columns = {"tweet_id": "id"}, inplace = True)
# Keep only the tweet and music_id columns
musicDf = musicDf[["id", "music_id"]]
# Drop duplicates
musicDf.drop_duplicates(inplace = True)
# Check
print("Shape:", musicDf.shape)
musicDf.sample()
# tweets with multiple music ids
musicDf.groupby("id").music_id.nunique().reset_index().query("music_id > 1").shape

Shape: (47497, 2)


<ipython-input-11-7410508ea0d8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musicDf.drop_duplicates(inplace = True)


(190, 2)

In [ ]:
# Update the main dataframe by adding music_id
tweetsDf = tweetsDf.merge(musicDf, on = ["id"], how = "left")
print("Shape after adding the music sessions", tweetsDf.shape)
# Update the unmatched music_id with 0
tweetsDf["music_id"] = tweetsDf["music_id"].fillna(0)
# Check
tweetsDf.sample()

Shape after adding the music sessions (15542629, 18)


,id,tweet_type,referenced_tweet_type,created_at,lang,disorder,author_id,text,cleaned_text,retweet_count,reply_count,like_count,quote_count,source,group,diagnosis_date,treatment_date,music_id
10042278,1445676059555348488,timeline,replied_to,2021-10-06 09:03:26,en,anxiety,4230507676,@BookwormVaught Sweet!,sweet!,0,0,0,0,Twitter for iPhone,1,2020-05-06,2021-11-21,0.0


In [ ]:
# Update the tweet type, set it to "treatment" wherever the music_id is not 0
tweetsDf.loc[tweetsDf["music_id"] != 0, "tweet_type"] = "treatment"
# Check
tweetsDf.sample()

,id,tweet_type,referenced_tweet_type,created_at,lang,disorder,author_id,text,cleaned_text,retweet_count,reply_count,like_count,quote_count,source,group,diagnosis_date,treatment_date,music_id
13339364,1393929230401130498,timeline,replied_to,2021-05-16 14:00:01,en,ptsd,19553548,@RoyalAirForce @RAFMusic Wing Commander Guy Gi...,wing commander guy gibson (in door of aircraft...,11,1,40,1,Twitter Web App,1,2021-03-26,2020-11-22,0.0


### Add Demographics
Before running this part make sure the demographics pipeline was run and all 3 demographics files were created successfully

In [ ]:
# Read the demographics user files
ageDf = pd.read_pickle(os.path.join(demographicsDir, "authorAge.pickle"))
genderDf = pd.read_pickle(os.path.join(demographicsDir, "authorGender.pickle"))
eduLevelDf = pd.read_pickle(os.path.join(demographicsDir, "authorEducationLevel.pickle"))
eduLevelDf.rename(columns = {"ari_grade": "edu_level"}, inplace = True)
# Check
print("Age df Shape:", ageDf.shape)
print("Gender df Shape:", genderDf.shape)
print("Education Level df Shape:", eduLevelDf.shape)

Age df Shape: (6536, 30)
Gender df Shape: (6536, 29)
Education Level df Shape: (6580, 6)


In [ ]:
# Check if the columns exist
if set(["age_group", "gender", "edu_level"]).issubset(set(tweetsDf.columns)):
  # drop & update
  # Merge the demographics data with the tweetsDf
  tweetsDf = tweetsDf.merge(ageDf[["author_id", "age_group"]], on = ["author_id"], how = "left")
  tweetsDf = tweetsDf.merge(genderDf[["author_id", "gender"]], on = ["author_id"], how = "left")
  tweetsDf = tweetsDf.merge(eduLevelDf[["author_id", "edu_level"]], on = ["author_id"], how = "left")
  tweetsDf.drop(["age_group", "gender", "edu_level"], axis = 1, inplace = True)
else: # Update
  # Merge the demographics data with the tweetsDf
  tweetsDf = tweetsDf.merge(ageDf[["author_id", "age_group"]], on = ["author_id"], how = "left")
  tweetsDf = tweetsDf.merge(genderDf[["author_id", "gender"]], on = ["author_id"], how = "left")
  tweetsDf = tweetsDf.merge(eduLevelDf[["author_id", "edu_level"]], on = ["author_id"], how = "left")
# Check
print("Shape after adding the demographics", tweetsDf.shape)

Shape after adding the demographics (15542629, 27)


### TensiStrenght Score
- Run the TensiStrenght Pipeline on timeline tweets with text
- Make sure the TensiStrength pipleline generated the files successfully before running this part

In [ ]:
# Read the tensiStrenght files (Depression)
depressionWithTensiDf = pd.read_pickle(os.path.join(tensiStrenghtDir, "depressionFullWithTensiScore.pickle"))
print("Shape:", depressionWithTensiDf.shape)
# Select the required columns only
depressionWithTensiDf = depressionWithTensiDf[["id", "author_id", "relax_score_org", "stress_score_org", "relax_score", "stress_score", "combined_score"]]

# Read the tensiStrenght files (Anxiety)
anxietyWithTensiDf = pd.read_pickle(os.path.join(tensiStrenghtDir, "anxietyFullWithTensiScore.pickle"))
print("Shape:", anxietyWithTensiDf.shape)
# Select the required columns only
anxietyWithTensiDf = anxietyWithTensiDf[["id", "author_id", "relax_score_org", "stress_score_org", "relax_score", "stress_score", "combined_score"]]

# Read the tensiStrenght files (PTSD)
ptsdWithTensiDf = pd.read_pickle(os.path.join(tensiStrenghtDir, "ptsdFullWithTensiScore.pickle"))
print("Shape:", ptsdWithTensiDf.shape)
# Select the required columns only
ptsdWithTensiDf = ptsdWithTensiDf[["id", "author_id", "relax_score_org", "stress_score_org", "relax_score", "stress_score", "combined_score"]]

# Combine the anxiety, depression, and PTSD TensiStrenght dataframes and remove duplicates based on id
tensiStrenghtDf = pd.concat([anxietyWithTensiDf, depressionWithTensiDf, ptsdWithTensiDf]) #, missingWithTensiDf
tensiStrenghtDf.drop_duplicates(subset = ["id"], inplace = True)
print("Tensi df Shape:", tensiStrenghtDf.shape)

# Merge with tweetsDf based on id
cols = ["id", "relax_score_org", "stress_score_org", "relax_score", "stress_score", "combined_score"]
# Check if the columns exist in the dataframe
if set(cols).issubset(set(tweetsDf.columns)):
  # drop
  tweetsDf.drop(["relax_score_org", "stress_score_org", "relax_score", "stress_score", "combined_score"], axis = 1, inplace = True)
else: # Update
  tweetsDf = tweetsDf.merge(tensiStrenghtDf[cols], on = ["id"], how = "left")
  print("Shape of tweets df after adding the tensiStrenght score", tweetsDf.shape)

Shape: (7272947, 28)
Shape: (3319097, 28)
Shape: (2930285, 28)
Tensi df Shape: (13522081, 7)
Shape of tweets df after adding the tensiStrenght score (15542629, 26)


#### Validate

In [ ]:
# Find anxiety/depression/PTSD tweets with no score
missingTensiDf = tweetsDf[(tweetsDf.disorder.isin(["depression", "anxiety", "ptsd"])) &
(tweetsDf.combined_score.isna()) &
(tweetsDf.lang == "en") &
(tweetsDf.cleaned_text.notna())]

missingCount = missingTensiDf.shape[0]
print("Missing Tensi scores:", missingCount)

# Pickle the resulting dataframe
if missingCount > 0:
  fileName = os.path.join(tensiStrenghtDir, "missingTensiDf.pickle")
  missingTensiDf.to_pickle(fileName)
  # Report
  print("Shape of the missing Tensi scores df:", missingTensiDf.shape)
  missingTensiDf.sample()

Missing Tensi scores: 0


### User/Authors Info

In [ ]:
# Read the data
controlUsers = pd.read_csv(os.path.join(authorsDir, "authors_control.csv"))
# Add the group column
controlUsers["group"] = 0
print("Control users shape:", controlUsers.shape)
treatmentUsers = pd.read_csv(os.path.join(authorsDir, "authors_treatment.csv"))
treatmentUsers["group"] = 1
print("Treatment users shape:", treatmentUsers.shape)

# Combine
usersDf = pd.concat([controlUsers, treatmentUsers])

# Remove the unnecessary columns
usersDf.drop(["anonymized_id", "matched_author_count", "matched_author_ids", "location", "username"], axis = 1, inplace = True)

# Fix the datatypes
usersDf["created_at"] = pd.to_datetime(usersDf["created_at"]).dt.tz_localize(None)
# find the account age
usersDf["account_age"] = (pd.to_datetime("today") - usersDf["created_at"]).dt.days/365.0
# Fill na
usersDf["account_age"].fillna(0, inplace = True)
toFill = ["followers_count", "following_count", "tweet_count"]
# Fill and convert to int
usersDf[toFill] = usersDf[toFill].fillna(0)
usersDf["followers_count"] = usersDf["followers_count"].astype("int64")
usersDf["following_count"] = usersDf["following_count"].astype("int64")
usersDf["tweet_count"] = usersDf["tweet_count"].astype("int64")

# Fill the rest of columns with nothing
usersDf.fillna("", inplace = True)

# find the length of the description field
usersDf["description_len"] = usersDf["description"].apply(lambda x: len(x.split()))

# Reorganize the columns
reOrg = ['id', 'created_at', "account_age", 'verified', 'name', 'description', "description_len", 'disorder', 'group', 'followers_count', 'following_count', 'tweet_count']
usersDf = usersDf[reOrg]
# Rename id to author_id
usersDf.rename({"id":"author_id", "created_at": "author_since"}, axis = 1, inplace = True)
# Change the author_id data type to str
usersDf["author_id"] = usersDf["author_id"].astype("str")
# Check
print("Full dataset shape:", usersDf.shape)

Control users shape: (4362, 16)
Treatment users shape: (2251, 16)
Full dataset shape: (6613, 12)


In [ ]:
# Add the user info to tweetsDf
tweetsDf = tweetsDf.merge(usersDf[["author_id", "author_since"]], on = ["author_id"], how = "left")
print("Shape after adding the user info", tweetsDf.shape)
# Check
tweetsDf.sample()

Shape after adding the user info (15542629, 27)


,id,tweet_type,referenced_tweet_type,created_at,lang,disorder,author_id,text,cleaned_text,retweet_count,...,music_id,age_group,gender,edu_level,relax_score_org,stress_score_org,relax_score,stress_score,combined_score,author_since
15116839,1321871917721264128,timeline,replied_to,2020-10-29 17:49:59,en,bipolar,294576982,@ABingham93 aye theyre more solid than ma hand 😂,aye theyre more solid than ma hand 😂,0,...,0.0,Gen-Z,Male,Middle/Elemntary School,NaN,NaN,NaN,NaN,NaN,2011-05-07 07:55:07


### Final Touches

In [ ]:
tweetsDf.sample()

,id,tweet_type,referenced_tweet_type,created_at,lang,disorder,author_id,author_since,text,cleaned_text,...,treatment_date,music_id,relax_score_org,stress_score_org,relax_score,stress_score,combined_score,age_group,gender,edu_level
1493569,1350110557085556746,timeline,original,2021-01-15 16:00:15,en,anxiety,1125082133893132291,2019-05-05 12:57:38,Not surprising https://t.co/DhAWiHnkma,not surprising,...,2021-03-13,0.0,1.0,-1.0,0.0,0.0,0.5,Gen-Z,Male,Middle/Elemntary School


In [ ]:
tweetsDf.dtypes

id                                int64
tweet_type                       object
referenced_tweet_type            object
created_at               datetime64[ns]
lang                             object
disorder                         object
author_id                        object
text                             object
cleaned_text                     object
retweet_count                     int64
reply_count                       int64
like_count                        int64
quote_count                       int64
source                           object
group                             int64
diagnosis_date                   object
treatment_date           datetime64[ns]
music_id                        float64
age_group                        object
gender                           object
edu_level                        object
relax_score_org                 float64
stress_score_org                float64
relax_score                     float64
stress_score                    float64


In [ ]:
# Final data types fixes
tweetsDf["diagnosis_date"] = pd.to_datetime(tweetsDf["diagnosis_date"])
# Fill na on referenced_tweet_type
tweetsDf["referenced_tweet_type"] = tweetsDf["referenced_tweet_type"].fillna("original")

In [ ]:
tweetsDf.columns

Index(['id', 'tweet_type', 'referenced_tweet_type', 'created_at', 'lang',
       'disorder', 'author_id', 'author_since', 'text', 'cleaned_text',
       'retweet_count', 'reply_count', 'like_count', 'quote_count', 'source',
       'group', 'diagnosis_date', 'treatment_date', 'music_id',
       'relax_score_org', 'stress_score_org', 'relax_score', 'stress_score',
       'combined_score', 'age_group', 'gender', 'edu_level'],
      dtype='object')

In [ ]:
# Reorganize the columns
cols = ['id', 'tweet_type', 'referenced_tweet_type', 'created_at', 'lang',
       'disorder', 'author_id', 'author_since', 'text', 'cleaned_text', 'retweet_count',
       'reply_count', 'like_count', 'quote_count', 'source', 'group',
       'diagnosis_date', 'treatment_date', 'music_id', 'relax_score_org',
       'stress_score_org', 'relax_score', 'stress_score', 'combined_score',
       'age_group', 'gender', 'edu_level']
tweetsDf = tweetsDf[cols]

### Save
Main and individual files

In [ ]:
# Save the main dataframe to parquet
fileName = os.path.join(tweetsDir, "processedTweets.parquet")
tweetsDf.to_parquet(fileName)

In [ ]:
# Save the anxiety data with Tensi Score
anxietyWithTensiScoreDf = tweetsDf[(tweetsDf.disorder == "anxiety")].drop_duplicates
fileName = os.path.join(tensiStrenghtDir, "anxietyFullWithTensiScore.pickle")
anxietyWithTensiScoreDf.to_pickle(fileName)
print("Shape:", anxietyWithTensiScoreDf.shape)

Shape: (3319097, 27)


In [ ]:
# Save the depression data with Tensi Score
depressionWithTensiScoreDf = tweetsDf[(tweetsDf.disorder == "depression")]
fileName = os.path.join(tensiStrenghtDir, "depressionFullWithTensiScore.pickle")
depressionWithTensiScoreDf.to_pickle(fileName)
print("Shape:", depressionWithTensiScoreDf.shape)

Shape: (7272947, 27)


In [ ]:
# Save the PTSD data with Tensi Score
ptsdWithTensiScoreDf = tweetsDf[(tweetsDf.disorder == "ptsd")]
fileName = os.path.join(tensiStrenghtDir, "ptsdFullWithTensiScore.pickle")
ptsdWithTensiScoreDf.to_pickle(fileName)
print("Shape:", ptsdWithTensiScoreDf.shape)

Shape: (2930285, 27)
